In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import json
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


# Función para crear la base de datos SQLite si no existe
def create_database():
    connection = sqlite3.connect('invoices.db')
    cursor = connection.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS invoices (
            id INTEGER PRIMARY KEY,
            customer_name TEXT NOT NULL,
            date TEXT NOT NULL,
            total_amount REAL NOT NULL
        )
    ''')

    connection.commit()
    connection.close()

if __name__ == "__main__":
    create_database()



# Carpeta donde se guardarán los archivos de texto
data_folder = 'Datos'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Rutas completas de los archivos dentro de la carpeta 'Datos'
CUSTOMER_FILE = os.path.join(data_folder, 'clientes.txt')
INVOICE_FILE = os.path.join(data_folder, 'facturas.txt')
DETAILS_FILE = os.path.join(data_folder, 'detalles_facturas.txt')
CUENTAS_FILE = os.path.join(data_folder, 'cuentas_corrientes.txt')

def load_data():
    global customer_id_map, invoices, cuentas_corrientes
    
    # Inicializar variables globales
    customer_id_map = {}
    invoices = []
    cuentas_corrientes = {}

    try:
        with open(CUSTOMER_FILE, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            customer_id_map = {}
            for line in lines:
                if line.startswith("ID:"):
                    parts = line.split(", Nombre: ")
                    customer_id = parts[0].split(": ")[1].strip()
                    customer_name = parts[1].strip()
                    customer_id_map[customer_name] = customer_id
    except FileNotFoundError:
        customer_id_map = {}

    try:
        with open(INVOICE_FILE, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            invoices = []
            invoice = {}
            for line in lines:
                if line.startswith("Número de factura:"):
                    if invoice:
                        invoices.append(invoice)
                    invoice = {}
                    parts = line.split(", Cliente: ")
                    invoice_number = parts[0].split(": ")[1].strip()
                    invoice['invoice_number'] = invoice_number
                    invoice_details = parts[1].split(", Fecha: ")
                    invoice['customer_name'] = invoice_details[0].strip()
                    invoice_date = invoice_details[1].split(", Total: $")[0].strip()
                    invoice['date'] = invoice_date
                    total_amount = invoice_details[1].split(", Total: $")[1].strip()
                    invoice['total_amount'] = float(total_amount)
                    invoice['detalles'] = []  # Inicializar detalles como lista vacía
            if invoice:
                invoices.append(invoice)
    except FileNotFoundError:
        invoices = []

    try:
        with open(DETAILS_FILE, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            current_invoice_number = None
            for line in lines:
                if line.startswith("Factura Número:"):
                    current_invoice_number = line.split(": ")[1].strip()
                elif line.startswith("  Descripción:"):
                    parts = line.split(", Cantidad: ")
                    descripcion = parts[0].split(": ")[1].strip()
                    cantidad = float(parts[1].split(", Precio unitario: $")[0].strip())
                    precio_unitario = float(parts[1].split(", Precio unitario: $")[1].split(", IVA: ")[0].strip())
                    iva = float(parts[1].split(", IVA: ")[1].split("%, Otros: $")[0].strip())
                    otros = float(parts[1].split("%, Otros: $")[1].strip())
                    detalle = {
                        'descripcion': descripcion,
                        'cantidad': cantidad,
                        'precio_unitario': precio_unitario,
                        'iva': iva,
                        'otros': otros
                    }
                    for invoice in invoices:
                        if invoice['invoice_number'] == current_invoice_number:
                            invoice['detalles'].append(detalle)
                            break
    except FileNotFoundError:
        pass

    try:
        with open(CUENTAS_FILE, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            cuentas_corrientes = {}
            customer_id = None
            cuenta = {}
            for line in lines:
                if line.startswith("ID Cliente:"):
                    if cuenta and customer_id:
                        cuentas_corrientes[customer_id] = cuenta
                    cuenta = {}
                    parts = line.split(", Nombre Cliente: ")
                    customer_id = parts[0].split(": ")[1].strip()
                    cuenta['customer_name'] = parts[1].strip()
                    cuenta['movimientos'] = []
                elif line.startswith("  Fecha:"):
                    movimiento = {}
                    parts = line.split(", Debe: ")
                    movimiento['fecha'] = parts[0].split(": ")[1].strip()
                    debe_haber = parts[1].split(", Haber: ")
                    movimiento['debe'] = float(debe_haber[0].strip())
                    debe_haber_detalle = debe_haber[1].split(", Detalle: ")
                    movimiento['haber'] = float(debe_haber_detalle[0].strip())
                    detalle_forma_pago = debe_haber_detalle[1].split(", Forma de pago: ")
                    movimiento['detalle'] = detalle_forma_pago[0].strip()
                    forma_pago_referencia = detalle_forma_pago[1].split(", Número de referencia: ")
                    movimiento['forma_pago'] = forma_pago_referencia[0].strip()
                    movimiento['numero_referencia'] = forma_pago_referencia[1].strip()
                    cuenta['movimientos'].append(movimiento)
                elif line.startswith("Saldo:"):
                    cuenta['saldo'] = float(line.split(": ")[1].strip())
            if cuenta and customer_id:
                cuentas_corrientes[customer_id] = cuenta
    except FileNotFoundError:
        cuentas_corrientes = {}

def save_data():
    with open(CUSTOMER_FILE, 'w', encoding='utf-8') as f:
        f.write("Clientes:\n")
        for customer_name, customer_id in customer_id_map.items():
            f.write(f"ID: {customer_id}, Nombre: {customer_name}\n")
    
    with open(INVOICE_FILE, 'w', encoding='utf-8') as f:
        f.write("Facturas:\n")
        for invoice in invoices:
            f.write(f"Número de factura: {invoice['invoice_number']}, Cliente: {invoice['customer_name']}, Fecha: {invoice['date']}, Total: ${invoice['total_amount']:.2f}\n")
    
    with open(CUENTAS_FILE, 'w', encoding='utf-8') as f:
        f.write("Cuentas corrientes:\n")
        for customer_id, cuenta in cuentas_corrientes.items():
            f.write(f"ID Cliente: {customer_id}, Nombre Cliente: {cuenta['customer_name']}\n")
            f.write("Movimientos:\n")
            for movimiento in cuenta['movimientos']:
                f.write(f"  Fecha: {movimiento['fecha']}, Debe: {movimiento['debe']}, Haber: {movimiento['haber']}, Detalle: {movimiento['detalle']}, Forma de pago: {movimiento['forma_pago']}, Número de referencia: {movimiento['numero_referencia']}\n")
            f.write(f"Saldo: {cuenta['saldo']}\n")
            f.write("-"*40 + "\n")

    with open(DETAILS_FILE, 'w', encoding='utf-8') as f:
        f.write("Detalles de Facturas:\n")
        for invoice in invoices:
            f.write(f"Factura Número: {invoice['invoice_number']}:\n")
            if 'detalles' in invoice:
                for detalle in invoice['detalles']:
                    f.write(f"  Descripción: {detalle['descripcion']}, Cantidad: {detalle['cantidad']}, Precio unitario: ${detalle['precio_unitario']:.2f}, IVA: {detalle['iva']}%, Otros: ${detalle['otros']:.2f}\n")
            else:
                f.write("  No se encontraron detalles para esta factura.\n")
            f.write(f"Total: ${invoice['total_amount']:.2f}\n")
            f.write("-"*40 + "\n")




def generate_customer_id(cliente):
    if cliente in customer_id_map:
        return customer_id_map[cliente]
    else:
        new_id = f"C{len(customer_id_map) + 1:04d}"
        customer_id_map[cliente] = new_id
        return new_id


def add_to_cuenta_corriente(factura):
    customer_id = factura["customer_id"]
    customer_name = factura["customer_name"]

    if customer_id not in cuentas_corrientes:
        cuentas_corrientes[customer_id] = {
            "customer_name": customer_name,
            "movimientos": [],
            "saldo": 0
        }

    nuevo_movimiento = {
        "fecha": factura["date"],
        "debe": factura["total_amount"],
        "haber": 0,
        "detalle": f"Factura {factura['invoice_number']}",
        "forma_pago": "N/A",
        "numero_referencia": factura["invoice_number"]
    }

    cuentas_corrientes[customer_id]["movimientos"].append(nuevo_movimiento)
    cuentas_corrientes[customer_id]["saldo"] += factura["total_amount"]




# Función para limpiar los campos de entrada
def clear_entry_fields(entries):
    for entry in entries.values():
        entry.delete(0, tk.END)

# Función para mostrar las facturas en el Listbox
def display_invoices(listbox, invoices):
    listbox.delete(0, tk.END)  # Limpiar cualquier contenido previo del Listbox
    for invoice in invoices:
        customer_name = invoice.get('customer_name', 'N/A')
        invoice_number = invoice.get('invoice_number', 'N/A')
        total_amount = invoice.get('total_amount', 'N/A')
        listbox.insert(tk.END, f"{customer_name} - {invoice_number} - ${total_amount:.2f}")

# Cargar los datos iniciales
load_data()


def search_customer_id_by_name(customer_name):
    return customer_id_map.get(customer_name)

def update_cuenta_corriente_listbox(customer_id):
    # Implementar según sea necesario para mostrar la cuenta corriente del cliente
    pass

def add_detail_row(frame, entries):
    row = len(entries["Detalles"]) + 1

    # Crear campos de entrada para descripción, cantidad, precio unitario y IVA
    entry_descripcion = ttk.Entry(frame, width=30)
    entry_descripcion.grid(row=row, column=1, padx=5, pady=5, sticky='we')
    entry_cantidad = ttk.Entry(frame, width=10)
    entry_cantidad.grid(row=row, column=2, padx=5, pady=5, sticky='we')
    entry_precio_unitario = ttk.Entry(frame, width=15)
    entry_precio_unitario.grid(row=row, column=3, padx=5, pady=5, sticky='we')
    entry_iva = ttk.Entry(frame, width=10)
    entry_iva.grid(row=row, column=4, padx=5, pady=5, sticky='we')
    entry_otros = ttk.Entry(frame, width=15)  # Nueva entrada para Otros
    entry_otros.grid(row=row, column=5, padx=5, pady=5, sticky='we')

    # Agregar entradas a la lista de detalles
    entries["Detalles"].append({
        "descripcion": entry_descripcion,
        "cantidad": entry_cantidad,
        "precio_unitario": entry_precio_unitario,
        "iva": entry_iva,
        "otros": entry_otros  # Incluir la entrada de Otros
    })


def add_invoice(entries, detalles):
    fecha = entries["Fecha"].get()
    numero_factura = entries["Número de factura"].get()
    cliente = entries["Cliente"].get().upper() 

    # Validar datos de entrada
    if not (fecha and numero_factura and cliente):
        messagebox.showwarning("Advertencia", "Por favor complete todos los campos.")
        return

    # Generar ID de cliente si no existe
    customer_id = generate_customer_id(cliente)

    # Crear factura y agregarla a la lista de facturas
    nueva_factura = {
        "date": fecha,
        "invoice_number": numero_factura,
        "customer_name": cliente,
        "customer_id": customer_id,
        "detalles": [],
        "total_amount": 0
    }

    monto_total_factura = 0.0

    for detalle in detalles:
        descripcion = detalle["descripcion"].get().upper()
        cantidad = float(detalle["cantidad"].get())
        precio_unitario = float(detalle["precio_unitario"].get())
        iva = float(detalle["iva"].get())
        otros = float(detalle["otros"].get())

        # Validar detalles de factura
        if not (descripcion and cantidad and precio_unitario and iva):
            messagebox.showwarning("Advertencia", "Por favor complete todos los campos de los detalles de la factura.")
            return

        subtotal = cantidad * precio_unitario
        total_con_iva = subtotal * (1 + iva / 100)
        monto_total_factura += total_con_iva + otros

        nuevo_detalle = {
            "descripcion": descripcion,
            "cantidad": cantidad,
            "precio_unitario": precio_unitario,
            "iva": iva,
            "otros": otros
        }
        nueva_factura["detalles"].append(nuevo_detalle)

    nueva_factura["total_amount"] = monto_total_factura

    invoices.append(nueva_factura)
    # Agregar a la cuenta corriente
    add_to_cuenta_corriente(nueva_factura)

    messagebox.showinfo("Información", "Factura agregada exitosamente.")

    # Actualizar la interfaz gráfica para mostrar la nueva factura
    display_invoices(listbox_invoices, invoices)
    total_label.config(text=f"Total: ${nueva_factura['total_amount']:.2f}")

    # Limpiar campos después de agregar la factura
    clear_entry_fields(entries)
    for detalle in detalles:
        clear_entry_fields(detalle)




# Función para agregar un pago
def add_payment(entries_payment):
    date = entries_payment["Fecha"].get().strip()
    Id_cliente = entries_payment["Id_cliente"].get().strip()
    customer_name = entries_payment["Cliente"].get().strip().upper()
    amount_str = entries_payment["Monto"].get().strip()
    payment_method = entries_payment["Forma de pago"].get().strip()
    reference_number = entries_payment["Número de Referencia"].get().strip()

    # Verificar que los campos no estén vacíos
    if not all([date, customer_name, amount_str, payment_method, reference_number]):
        messagebox.showwarning("Advertencia", "Todos los campos son obligatorios")
        return

    # Validar el formato del monto
    try:
        amount = float(amount_str.replace('$', '').replace(',', ''))
    except ValueError:
        messagebox.showwarning("Advertencia", "Formato de monto incorrecto. Use el formato '$#,##0.00'")
        return

    # Buscar el ID del cliente por su nombre
    customer_id = search_customer_id_by_name(customer_name)

    if customer_id is None:
        messagebox.showwarning("Advertencia", f"No se encontró ID de cliente para el nombre: {customer_name}")
        return

    # Registrar el pago en la cuenta corriente del cliente
    if customer_id in cuentas_corrientes:
        movimiento = {
            'fecha': date,
            'debe': 0,
            'haber': amount,
            'detalle': f"Pago - {payment_method}",
            'forma_pago': payment_method,
            'numero_referencia': reference_number,
            'customer_id': customer_id  # Agregar el ID del cliente al movimiento
        }
        cuentas_corrientes[customer_id]['movimientos'].append(movimiento)
        cuentas_corrientes[customer_id]['saldo'] -= amount
        messagebox.showinfo("Información", "Pago registrado exitosamente")
    else:
        messagebox.showwarning("Advertencia", f"No se encontró una cuenta corriente para el ID de cliente: {customer_id}")

    # Limpiar los campos de entrada después de agregar el pago
    clear_entry_fields(entries_payment)
    # Actualizar la visualización de la planilla
    display_planilla()

# Función para agregar una transacción a la cuenta corriente
def add_to_cuenta_corriente(nueva_factura):
    customer_id = nueva_factura["customer_id"]
    total_amount = nueva_factura["total_amount"]

    if customer_id in cuentas_corrientes:
        cuentas_corrientes[customer_id]["movimientos"].append({
            "fecha": nueva_factura["date"],
            "debe": total_amount,
            "haber": 0,
            "detalle": f"Factura #{nueva_factura['invoice_number']}",
            "forma_pago": "N/A",
            "numero_referencia": "N/A"
        })
        cuentas_corrientes[customer_id]["saldo"] += total_amount
    else:
        cuentas_corrientes[customer_id] = {
            "customer_name": nueva_factura["customer_name"],
            "movimientos": [{
                "fecha": nueva_factura["date"],
                "debe": total_amount,
                "haber": 0,
                "detalle": f"Factura #{nueva_factura['invoice_number']}",
                "forma_pago": "N/A",
                "numero_referencia": "N/A"
            }],
            "saldo": total_amount
        }


# Función para mostrar la planilla
def display_planilla():
    text_widget.config(state=tk.NORMAL)
    text_widget.delete('1.0', tk.END)  # Borra el contenido actual del text widget
    for customer_id, cuenta in cuentas_corrientes.items():
        text_widget.insert(tk.END, f"ID Cliente: {customer_id}\n")
        text_widget.insert(tk.END, f"Nombre Cliente: {cuenta['customer_name']}\n")
        text_widget.insert(tk.END, "Movimientos:\n")
        for movimiento in cuenta['movimientos']:
            text_widget.insert(tk.END, f"  Fecha: {movimiento['fecha']}, Debe: {movimiento['debe']}, Haber: {movimiento['haber']}, Detalle: {movimiento['detalle']}, Forma de pago: {movimiento['forma_pago']}, Número de referencia: {movimiento['numero_referencia']}\n")
        text_widget.insert(tk.END, f"Saldo: {cuenta['saldo']}\n")
        text_widget.insert(tk.END, "-"*40 + "\n")

    text_widget.config(state=tk.DISABLED)

 
# Función para buscar el ID del cliente por su nombre# Función para buscar el ID del cliente por su nombre
def search_customer_id_by_name(customer_name):
    # Intenta buscar el ID directamente en el diccionario de cuentas corrientes
    for key, value in cuentas_corrientes.items():
        if value['customer_name'].upper() == customer_name.upper():  # Comparación sin considerar mayúsculas o minúsculas
            return key  # Aquí 'key' sería el ID del cliente
    
    # Si no se encontró en ninguna parte, retornar None
    return None

# Función para mostrar cambios en la planilla de cuentas corrientes
def mostrar_cambios():
    text_widget.config(state=tk.NORMAL)
    text_widget.delete('1.0', tk.END)  # Borra el contenido actual del text widget
    for customer_id, cuenta in cuentas_corrientes.items():
        text_widget.insert(tk.END, f"ID Cliente: {customer_id}\n")
        text_widget.insert(tk.END, f"Nombre Cliente: {cuenta['customer_name']}\n")
        text_widget.insert(tk.END, "Movimientos:\n")
        saldo_acumulado = 0
        for movimiento in cuenta['movimientos']:
            text_widget.insert(tk.END, f"  Fecha: {movimiento['fecha']}, Debe: {movimiento['debe']}, Haber: {movimiento['haber']}, Detalle: {movimiento['detalle']}, Forma de pago: {movimiento['forma_pago']}, Número de referencia: {movimiento['numero_referencia']}\n")
            saldo_acumulado += movimiento['debe'] - movimiento['haber']
        text_widget.insert(tk.END, f"Saldo acumulado: {saldo_acumulado}\n")
        text_widget.insert(tk.END, "-"*40 + "\n")

    text_widget.config(state=tk.DISABLED)


# Función para limpiar el campo de texto
def limpiar_campo():
    text_widget.config(state=tk.NORMAL)
    text_widget.delete('1.0', tk.END)
    text_widget.config(state=tk.DISABLED)






# Crear la ventana principal
root = tk.Tk()
root.title("Gestión de Facturas y Pagos")
# Función para manejar el cambio de tamaño de la ventana
def resize_event(event):
    # Aquí puedes agregar acciones adicionales al cambiar el tamaño de la ventana
    pass

root.bind("<Configure>", resize_event)

# Crear pestañas para facturas y pagos
tab_control = ttk.Notebook(root)
tab_facturas = ttk.Frame(tab_control)
tab_pagos = ttk.Frame(tab_control)
tab_planilla = ttk.Frame(tab_control) 
tab_control.add(tab_facturas, text="Facturas")
tab_control.add(tab_pagos, text="Pagos")
tab_control.add(tab_planilla, text="Planilla de Cuentas Corrientes", padding=(10, 1))  
tab_control.pack(expand=1, fill="both")


# Crear marco de entrada para facturas
frame_factura_entry = ttk.LabelFrame(tab_facturas, text="Agregar Factura")
frame_factura_entry.pack(padx=10, pady=10, fill="x")

# Campos de entrada para los datos de la factura
entries_factura = {
    "Fecha": ttk.Entry(frame_factura_entry),
    "Número de factura": ttk.Entry(frame_factura_entry),
    "Cliente": ttk.Entry(frame_factura_entry)
}
entries_factura["Fecha"].grid(row=0, column=1, padx=5, pady=5, sticky='we')
entries_factura["Número de factura"].grid(row=1, column=1, padx=5, pady=5, sticky='we')
entries_factura["Cliente"].grid(row=2, column=1, padx=5, pady=5, sticky='we')

ttk.Label(frame_factura_entry, text="Fecha").grid(row=0, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_factura_entry, text="Número de factura").grid(row=1, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_factura_entry, text="Cliente").grid(row=2, column=0, padx=5, pady=5, sticky='w')

# Marco de detalles de la factura
frame_factura_details = ttk.LabelFrame(tab_facturas, text="Detalles de la Factura")
frame_factura_details.pack(padx=10, pady=10, fill="x")

entries_detalles = {
    "Detalles": []
}
# Encabezados de los detalles
ttk.Label(frame_factura_details, text="Descripción").grid(row=0, column=1, padx=5, pady=5)
ttk.Label(frame_factura_details, text="Cantidad").grid(row=0, column=2, padx=5, pady=5)
ttk.Label(frame_factura_details, text="Precio Unitario").grid(row=0, column=3, padx=5, pady=5)
ttk.Label(frame_factura_details, text="IVA (%)").grid(row=0, column=4, padx=5, pady=5)
ttk.Label(frame_factura_details, text="Otros").grid(row=0, column=5, padx=5, pady=5)


# Botón para agregar una nueva fila de detalles
ttk.Button(frame_factura_details, text="Agregar Detalle", command=lambda: add_detail_row(frame_factura_details, entries_detalles)).grid(row=0, column=0, padx=5, pady=5, sticky='we')

# Botón para agregar la factura
style = ttk.Style()
style.configure('Custom.TButton', background='#52BE80', foreground='green')
ttk.Button(frame_factura_entry, text="Agregar Factura", style='Custom.TButton', command=lambda: add_invoice(entries_factura, entries_detalles["Detalles"])).grid(row=3, columnspan=3, padx=5, pady=5)


# Crear un Listbox para mostrar las facturas
listbox_invoices = tk.Listbox(tab_facturas, height=5, width=100)
listbox_invoices.pack(padx=10, pady=10, fill="both", expand=True)

# Mostrar las facturas en el Listbox
display_invoices(listbox_invoices, invoices)



# Crear marco de entrada para pagos
frame_payment_entry = ttk.LabelFrame(tab_pagos, text="Registrar Pago")
frame_payment_entry.pack(padx=10, pady=10, fill="x")

# Campos de entrada para los datos del pago
entries_payment = {
    "Fecha": ttk.Entry(frame_payment_entry),
    "Id_cliente": ttk.Entry(frame_payment_entry),
    "Cliente": ttk.Entry(frame_payment_entry),
    "Monto": ttk.Entry(frame_payment_entry),
    "Forma de pago": ttk.Entry(frame_payment_entry),
    "Número de Referencia": ttk.Entry(frame_payment_entry)
}

entries_payment["Fecha"].grid(row=0, column=1, padx=5, pady=5, sticky='we')
entries_payment["Id_cliente"].grid(row=1, column=1, padx=5, pady=5, sticky='we')
entries_payment["Cliente"].grid(row=2, column=1, padx=5, pady=5, sticky='we')
entries_payment["Monto"].grid(row=3, column=1, padx=5, pady=5, sticky='we')
entries_payment["Forma de pago"].grid(row=4, column=1, padx=5, pady=5, sticky='we')
entries_payment["Número de Referencia"].grid(row=5, column=1, padx=5, pady=5, sticky='we')

ttk.Label(frame_payment_entry, text="Fecha de Pago").grid(row=0, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_payment_entry, text="Id_cliente").grid(row=1, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_payment_entry, text="Cliente").grid(row=2, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_payment_entry, text="Monto").grid(row=3, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_payment_entry, text="Forma de pago").grid(row=4, column=0, padx=5, pady=5, sticky='w')
ttk.Label(frame_payment_entry, text="Número de Referencia").grid(row=5, column=0, padx=5, pady=5, sticky='w')

# Botón para agregar el pago
style.configure('Custom.TButton', background='#52BE80', foreground='green')
ttk.Button(frame_payment_entry, text="Agregar Pago", style='Custom.TButton', command=lambda: add_payment(entries_payment)).grid(row=6, columnspan=2, padx=5, pady=5)

# Crear el widget de texto para mostrar la planilla con la barra de desplazamiento
text_widget = tk.Text(tab_planilla)
text_widget.pack(padx=10, pady=10, fill="both", expand=True)

# Crear una barra de desplazamiento vertical
scrollbar = ttk.Scrollbar(tab_planilla, orient=tk.VERTICAL, command=text_widget.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Asociar la barra de desplazamiento al widget de texto
text_widget.config(yscrollcommand=scrollbar.set)

# Frame para mostrar el total de la factura
frame_total_factura = ttk.LabelFrame(tab_facturas, text="Total de la Factura")
frame_total_factura.pack(padx=10, pady=10, fill="x")
total_label = ttk.Label(frame_total_factura, text="Total: $0.00")
total_label.pack(padx=10, pady=10)

# Botón para mostrar cambios en la planilla
boton_mostrar = tk.Button(tab_planilla, text="Mostrar Cambios", command=mostrar_cambios)
boton_mostrar.pack(padx=1, pady=1)

# Botón para limpiar el campo de texto
boton_limpiar = tk.Button(tab_planilla, text="Limpiar Campo", command=limpiar_campo)
boton_limpiar.pack(padx=1, pady=1)


def mostrar_cc():
    # Conectar a la base de datos
    conexion = sqlite3.connect('invoices.db')
    cursor = conexion.cursor()
    
    # Consulta para obtener los clientes con saldo pendiente (saldo diferente de 0)
    cursor.execute("SELECT customer_name, total_amount FROM invoices WHERE ABS(total_amount) != 0")
    resultados = cursor.fetchall()
    
    # Depurar los resultados
    print(resultados)  # Verifica los resultados en la consola
    
    # Limpiar el Text widget antes de mostrar los cambios
    text_deudores.delete(1.0, tk.END)
    
    # Mostrar los resultados en el Text widget
    if resultados:
        for cliente, saldo in resultados:
            text_deudores.insert(tk.END, f"{cliente}: ${saldo}\n")
    else:
        text_deudores.insert(tk.END, "No hay clientes con saldo pendiente.\n")
    
    # Cerrar la conexión a la base de datos
    conexion.close()


# Botón para mostrar los clientes deudores
boton_cc = tk.Button(tab_planilla, text="Mostrar Clientes Deudores", command=mostrar_cc)
boton_cc.pack(padx=1, pady=1)

# Crear un Text widget para mostrar los clientes deudores
text_deudores = tk.Text(tab_planilla, height=10, width=100)
text_deudores.pack(padx=1, pady=1)






# Función para manejar el cierre de la aplicación
def on_closing():
    save_data()
    root.destroy()

# Guardar datos al cerrar la aplicación
root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()

[]
